# Numerical Integration

[Acknowledgments](#Acknowledgments)

Many physics problems require us to compute integrals. In this notebook we will explore numerical integration via several different methods, practicing our plotting and numpy skills along the way.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## A simple definite integral

We will start with the equation

$$ f(x) = x^3 - 3x + 5 $$

which you can, of course, integrate analytically. 

&#128310; *By hand* (i.e., not by Mathematica, not by Google, etc.) evaluate the integral of $f(x)$ from $a = 0$ to $b = 2$. Report your answer here.

&#128309; To integrate our function numerically, we will start by writing a function `f` that evaluates the expression $f(x)$. Write your function below.

A crude way to evaluate our integral numerically would be to sum up N rectangles centered on $f(x_i)$, for $x_i$ in the range $[a, b]$. 

In [ ]:
N = 5
a = 0.0
b = 2.0
h = (b-a)/N # step size for N steps from a to b

# Evaluate the integral at x = a
s = f(a)

# For each step k, evaluate the integral at x_i
for k in range(1, N+1):
    x_i = a + k*h

    # The += operator is handy for accumulating values. a += 2 is equivalent to a = a + 2.
    s += f(x_i)

integral_rectangles = s*h
print(integral_rectangles)

&#128310; Comment on the difference between your analytic solution and the numerical solution evaluated by summing N rectangles. Is the numerical solution an overestimate or an underestimate of the true value? Comment on whether you expected this. *Hint: think about the shape of the equation you are integrating in the range you are integrating it.*

## The Trapezoidal Rule

The trapezoidal rule is a slight modification to the above that improves the accuracy of our numerical integration. Instead of summing rectangles, let's sum trapezoids. If we divide the interval from $a$ to $b$ into $N$ slices, each slice has width $(b - a)/N$. The right-hand side of the $k^\mathrm{th}$ slice falls at $a + kh$, and the left-hand side falls at $a + hk - h = (k-1)h$. Thus the area of the trapezoid for this slice is 

$$A_k = \frac{1}{2}h \left[f(a + (k-1)h) + f(a + kh)\right]$$

This approximates the area under one slice of our function as a trapezoid. Extending this to the entire integral from $a$ to $b$, we find

$$\int_a^b f(x) \approx \frac{1}{2}h \sum_{k = 1}^{N} \left[f(a + (k-1)h) + f(a + kh)\right] = h\left[\frac{1}{2}f(a) + \frac{1}{2}f(b) + \sum_{k=1}^{N-1}f(a + kh)\right]$$



&#128309; Use the trapezoidal rule to integrate your function `f(x)` using a for loop through each slice `k`.

&#128310; Comment on the difference between your analytic solution to the integral and your numerical solution. What are the relevant sources of error in your computational solution? Try different values for `N` and comment on how the solution changes.

&#128309; Numpy has a function called `trapz`. Use it to numerically integrate and compare to your own trapezoidal rule implementation. Do you get the same output? 

## Replacing For Loops with Numpy Arrays

In the calculation we did above, we stepped through each item in our sum via a for loop. In previous notebooks, you may have done something similar, and if you wanted to store the output at each step (e.g., the cumulative sum as a function of number of steps), you may think of using a list to append new values at every step. These are operations that are available in pure Python, but having to loop through data and perform operations step by step can be quite inefficient. 

Replacing for loops with array operations can be a powerful way to speed up our code. Speeding up our code, in turn, allows us to tackle much larger datasets than we otherwise could. Writing calculations as manipulations of arrays, rather than as loops through individual elements, is called **vectorization**. Numpy allows us to do array-based calculations that are highly optimized and, under the hood, are written in compiled languages like C. Judiciously replacing some of your for loops with numpy arrays may take some getting used to, but it will be worth it!

&#128309; Let's test this by implementing two functions that each return the square of the first 1000 integers. The first will be implemented using a for loop and a list, and the second will be implementing using numpy arrays.

In [ ]:
# Write a function func1 that loops through 1000 values and appends each value^2 to a list.


# Write a function func2 that uses numpy to create an array of 1000 values and returns the array squared.


We can evaluate the performance of our two functions using the built-in, "magic" function `timeit`. Run the cell below.

In [ ]:
%timeit func1()
%timeit func2()

&#128310; What sort of speedup in performance did `func2` achieve relative to `func1`? Discuss your results. 

&#128309; Rewrite your implementation of the trapezoidal rule using numpy arrays. Hint: you do not need to rewrite the function `f`. You will replace your for loop of steps with a numpy array, and apply `f` to that array. You may want to make use of the numpy function `linspace` that we encountered in the Introduction notebook.

While you are getting the hang of array-based programming, it will often be helpful to code up what you want to do with for loops, or write for-loop-based pseudocode, and then translate to numpy arrays. 

## Visualizing our Numerical Integrals

To visualize our numerical integrals, we will first plot the function $f(x)$ by computing `f(x)` on a finely spaced grid of $x$ values. Then we will visualize our trapezoidal rule approximation of the integral using the function `plt.fill_between`. Put your code between the commented code outline below.

In [ ]:
# Define a finely spaced grid of x values for plotting purposes. 
# The range of these values should be somewhat larger than your bounds of integration [a, b].


# Plot the function f(x) evaluated on your finely spaced grid. 


# Plot your trapezoidal approximation to the integral from a=0 to b=2, using N = 5.
# To do this, you will define the points where your trapezoidal approximation is evaluated, and then use plt.fill_between.


## Simpson's Rule

Let's say we find ourselves needing to evaluate an integral with a higher level of accuracy than we get with the trapezoidal rule. The trapezoidal rule approximates the area under our curve with a series of line segments. What if we instead approximated each "chunk" with curves? Simpson's rule is precisely this: we'll fit a quadratic curve, rather than a straight line, to each chunk. To approximate a chunk with a quadratic, we'll need 3 points at a time, rather than 2. 

Let's walk through the calculation of a single chunk of integral with Simpson's rule. We will integrate $f(x)$ for a chunk centered at $x = 0$ that extends from $x = -h$ to $x = h$. If we fit a quadratic $A x^2 + Bx + C$ through these three points, that quadratic definitionally has:

$$f(-h) = Ah^2 -Bh + C$$
$$f(0) = C$$
$$f(h) = Ah^2 + Bh + C$$

Take a moment to convince yourself of the above on a piece of scrap paper, and show that solving the above equations simultaneously gives

$$A = \frac{1}{h^2} \left[\frac{1}{2} f(-h) - f(0) + \frac{1}{2}f(h)\right]$$
$$B = \frac{1}{2h} \left[f(h) - f(-h) \right]$$
$$C = f(0)$$

Integrate the quadratic from $-h$ to $h$ and plug in, and we find

$$\int_{-h}^{h} (Ax^2 + Bx + C)dx = \frac{2}{3}Ah^3 + 2Ch = \frac{1}{3}h\left[f(-h) + 4f(0) + f(h)\right]$$

Congratulations! You have just derived **Simpson's Rule**, a higher-order numerical integration scheme that is quite useful: fairly accurate while still being simple and fast. Before we code this up, there is one more thing to notice: when we chunk our integral up into $N$ regions, Simpson's Rule fits a quadratic to two chunks at a time -- in other words, $N$ needs to be even because we step by two's through the integral. The Simpson's Rule approximation of the entire integral is

$$\int_a^b f(x) \approx \frac{1}{3} h \left[ f(a) + f(b) + 4 \sum_{1,~\mathrm{odd}~k}^{N -1} f(a + kh) + 2 \sum_{2,~\mathrm{even}~k}^{N-2} f(a + kh) \right]$$



&#128309; Write a function called `simpsons` that integrates a given function via Simpson's Rule. 

&#128309; Once again, integrate your function `f(x)` from 0 to 2, this time using your function `simpsons`.

&#128310; Comment on the integral evaluated with Simpson's Rule. For a given `N`, how does the integral evaluated with the Trapezoidal Rule compare to the integral evaluated with Simpson's Rule? 

&#128309; Define a new function to integrate. Make it something funky and wiggly! Plot your function. Choose your bounds of integration and integrate your function using both the trapezoidal rule and Simpson's rule, experimenting with multiple step sizes. 

## Application to the diffraction limit of a telescope

Telescope observations have a fundamental resolution limit set by the diffraction of light. When light with wavelength $\lambda$ passes through the circular aperature of a telescope with diameter $D$, the intensity of light in the focal plane is described by a diffraction pattern (also called the point spread function of diffraction). The intensity on the focal plane follows the pattern

$$ I(r) = \left(\frac{2 J_1(k r)}{kr} \right)^2 $$

where $k = \frac{D \pi}{\lambda}$, $r$ is angular the distance from the center of the focal plane in radians, and $J_1(x)$ is a Bessel function given by

$$ J_m (x) = \frac{1}{\pi} \int_0^\pi \cos(m\theta - x\sin \theta) d\theta $$

&#128309; Write a function `J(m, x)` that calculates the value of $J_m(x)$ using Simpson's rule with N = 1000 points. 

&#128309; Use your function to make a single plot showing the Bessel functions $J_0$, $J_1$, and $J_2$ as a function of $x$ from $x=0$ to $x=20$. Label your Bessel functions using `plt.legend`.

&#128309; Import `scipy` and plot one of your Bessel functions using `scipy.special.jv` (scipy's implementation of the Bessel function). Plot the Bessel function computed with your function using a solid linestyle, and overplot scipy's version of the Bessel function using a dashed linestyle (`ls = '--'`). Do the results agree? 

&#128309; Use your implementation of the Bessel function to compute and plot the 2D intensity distribution of the circular diffraction pattern of a point source with $\lambda = 500$ nm in a $2 \mu\mathrm{m}$ x $2 \mu\mathrm{m}$ square region of the focal plane, for a telescope with $D = 2$ meters.

**Hint:** Be sure to pick a unit to work in for this problem and stick with it. I recommend meters. The comments below will help you structure your code. 

In [ ]:
# Define a grid of values representing coordinates in your focal plane using np.meshgrid. 


# Define the center of your focal plane. 


# Compute a square array that represents the distance from the center of the focal plane for every point in your focal plane.


# Evaluate the equation above for your array of distances from the center of the focal plane. 


# Plot your diffraction pattern using plt.pcolormesh.
# Add a colorbar using plt.colorbar()
# Note: the central peak is quite bright, so you may need to play with your colorbar limits. 
# plt.colormesh (and plt.imshow) take options vmin and vmax that set the values to map to the extremes of your colorbar. 
# I used vmax=0.02. 




If you took Physics 15 or another introductory astrophysics course, you learned a simple approximation for the "diffraction-limited" resolution of a telescope with diameter $D$ observing at wavelength $\lambda$: 

$$\theta = 1.22\frac{\lambda}{D}$$

&#128309; Use `plt.Circle` to plot a circle with that radius on your plot, and comment on the results. 

### Optional fun: 3D plotting 

Play around with the code below to visualize your diffraction pattern in 3D. 

In [ ]:
ax = plt.figure().add_subplot(projection='3d')
ax.plot_surface(your_x_variable, your_y_variable, your_amplitude_variable, edgecolor='royalblue', lw=0.5, alpha=0.3)


## Acknowledgments

S.E. Clark 2024, with trapezoidal and Simpson's rules discussions and diffraction limit problem all either inspired by or adapted from Newman 2013.
